In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
MODEL_PATH = 'best_emotion_model.keras' 
CLASS_NAMES = ['Angry', 'Fear', 'Happy', 'Sad', 'Suprise']
IMG_SIZE = 224
FACE_CASCADE_PATH = 'haarcascade_frontalface_default.xml'

In [3]:
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Success")
except Exception as e:
    print(f"Error: {e}")
    exit()

face_classifier = cv2.CascadeClassifier(FACE_CASCADE_PATH)
if face_classifier.empty():
    print("Error: File haarcascade tidak ditemukan!")
    exit()

Success


In [4]:
def start_recognition_video():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Tidak dapat mengakses kamera.")
        return

    print("Kamera dimulai. Tekan 'q' untuk keluar.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(
            gray_frame, 
            scaleFactor=1.3, 
            minNeighbors=5, 
            minSize=(30, 30)
        )

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            roi_face = frame[y:y + h, x:x + w]
            if roi_face.size == 0:
                continue

            roi_face = cv2.cvtColor(roi_face, cv2.COLOR_BGR2RGB)
            roi_face = cv2.resize(roi_face, (IMG_SIZE, IMG_SIZE))
            img_array = tf.keras.utils.img_to_array(roi_face)
            img_array = tf.expand_dims(img_array, 0)
            img_array = preprocess_input(img_array)


            predictions = model.predict(img_array, verbose=0)
            predicted_index = np.argmax(predictions[0])
            predicted_label = CLASS_NAMES[predicted_index]
            confidence = np.max(predictions[0]) * 100

            label_text = f"{predicted_label} ({confidence:.1f}%)"
            
            cv2.putText(frame, label_text, (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow('Real-time Emotion Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
if __name__ == "__main__":
    start_recognition_video()

Kamera dimulai. Tekan 'q' untuk keluar.
